In [1]:
import pandas as pd
import numpy as np
import spacy as sp
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.language import Language
from keras.preprocessing.text import Tokenizer
from spacy.lang.en import English


In [2]:
data = pd.read_json(r"CS663_Project_Data.json", orient='index')
data = data

In [3]:
df = data.copy()
df

,invocation,cmd
1,"Copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...
2,"Display all lines containing ""IP_MROUTE"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE
3,Display current running kernel's compile-time ...,cat /boot/config-`uname -r`
4,"Find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*
5,"Look for any instance of ""HIGHMEM"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`
...,...,...
10343,using exec in find command to dispaly the sear...,find . ... -exec cat {} \; -exec echo \;
10344,verbosely create intermediate directoriy tmp a...,mkdir -pv /tmp/boostinst
10345,view the manual page of find,man find
10346,"wait 2 seconds and then print ""hello""","echo ""hello `sleep 2 &`"""


In [4]:
from bashlint import data_tools, lint

df['cmd_ast'] = df['cmd'].apply(lint.normalize_ast)

Setting bashlex grammar using file: /Users/marcos/Desktop/School/Spring 2023/CS663/CS663-Final-Project/bashlint/grammar/grammar100.txt
Bashlint grammar set up (148 utilities)



In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10347 entries, 1 to 10347
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   invocation  10347 non-null  object
 1   cmd         10347 non-null  object
 2   cmd_ast     10347 non-null  object
dtypes: object(3)
memory usage: 323.3+ KB


In [6]:
drop = []
for i in range(1,len(df['cmd'])):
    try:
        df['cmd'][i] = data_tools.ast2template(df['cmd_ast'][i])
    except:
        drop.append(i)

df.drop(drop, inplace=True)

In [7]:
df

,invocation,cmd,cmd_ast
1,"Copy loadable kernel module ""mymodule.ko"" to t...",cp File $( uname -r ),<bashlint.nast.Node object at 0x7f815917abe0>
2,"Display all lines containing ""IP_MROUTE"" in th...",cat $( uname -r ) | grep Regex,<bashlint.nast.Node object at 0x7f815916eb20>
3,Display current running kernel's compile-time ...,cat $( uname -r ),<bashlint.nast.Node object at 0x7f8159143820>
4,"Find all loadable modules for current kernel, ...",find Path $( uname -r ) -regex Regex,<bashlint.nast.Node object at 0x7f81591435e0>
5,"Look for any instance of ""HIGHMEM"" in the curr...",grep Regex $( uname -r ),<bashlint.nast.Node object at 0x7f8159198400>
...,...,...,...
10343,using exec in find command to dispaly the sear...,find Path Path -exec cat {} \; -exec echo \;,<bashlint.nast.Node object at 0x7f815bca2340>
10344,verbosely create intermediate directoriy tmp a...,mkdir -p -v Directory,<bashlint.nast.Node object at 0x7f815bc5f7c0>
10345,view the manual page of find,man Regex,<bashlint.nast.Node object at 0x7f815b1571f0>
10346,"wait 2 seconds and then print ""hello""",echo $( sleep Timespan ),<bashlint.nast.Node object at 0x7f815bc47880>


In [18]:
from spacy.lang.en.stop_words import STOP_WORDS

def remove_stop_words(text):
    
    nlp = English()    
    
    text = text.lower()
    
    my_doc = nlp(text)

    # Create list of word tokens
    token_list = []
    for token in my_doc:
        token_list.append(token.text)
        
    filtered_sentence = []
    
    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
            
    return ' '.join(filtered_sentence)

In [15]:
def lemmatize_w2v(row):
    nlp = English()
    pipeline = ["lemmatizer"]
    for name in pipeline:
        nlp.add_pipe(name)
    nlp(row)

In [10]:
'''
from spacy.tokens import Doc


nlp = sp.load('en_core_web_sm')

@Language.component('clean_inv')
def clean_inv(doc):
    # Create a new list of filtered tokens
    filtered_tokens = []

    # Iterate over the tokens in the input document
    for token in doc:
        # Check conditions for token filtering
        if (not token.is_punct and not token.is_space and not token.is_digit and
            not token.like_num and not token.is_currency and not token.is_stop):

            # Append the lowercase version of the token's text to the filtered_tokens list
            filtered_tokens.append(token.lower_)

    # Create a new Doc object from the filtered tokens and return it
    return Doc(doc.vocab, words=filtered_tokens)

nlp.add_pipe('clean_inv', name='clean_inv', last=True)

def tokenize_inv(inv):
    doc = nlp(inv)
    tokens = [token.text for token in doc]
    return ' '.join(tokens)
    '''

"\nfrom spacy.tokens import Doc\n\n\nnlp = sp.load('en_core_web_sm')\n\n@Language.component('clean_inv')\ndef clean_inv(doc):\n    # Create a new list of filtered tokens\n    filtered_tokens = []\n\n    # Iterate over the tokens in the input document\n    for token in doc:\n        # Check conditions for token filtering\n        if (not token.is_punct and not token.is_space and not token.is_digit and\n            not token.like_num and not token.is_currency and not token.is_stop):\n\n            # Append the lowercase version of the token's text to the filtered_tokens list\n            filtered_tokens.append(token.lower_)\n\n    # Create a new Doc object from the filtered tokens and return it\n    return Doc(doc.vocab, words=filtered_tokens)\n\nnlp.add_pipe('clean_inv', name='clean_inv', last=True)\n\ndef tokenize_inv(inv):\n    doc = nlp(inv)\n    tokens = [token.text for token in doc]\n    return ' '.join(tokens)\n    "

In [19]:
df['inv_tokens'] = df['invocation'].apply(remove_stop_words)

KeyboardInterrupt: 

In [16]:
df['inv_tokens'] = df['invocation'].apply(lemmatize_w2v)
df

ValueError: [E1004] Missing lemmatizer table(s) found for lemmatizer mode 'rule'. Required tables: ['lemma_rules']. Found: []. Maybe you forgot to call `nlp.initialize()` to load in the data?

In [ ]:
'''
invocations = df['inv_tokens'].values
commands = ["startseq " + cmd + " endseq" for cmd in df['cmd'].values]

# Define the maximum number of words to keep based on word frequency
max_words = 10000

# Create a tokenizer for invocations and fit it on the tokenized text
inv_tokenizer = Tokenizer(num_words=max_words)
inv_tokenizer.fit_on_texts(invocations)

# Create a tokenizer for commands and fit it on the tokenized text
cmd_tokenizer = Tokenizer(num_words=max_words)
cmd_tokenizer.fit_on_texts(commands)

# Convert the tokenized text to sequences
inv_sequences = inv_tokenizer.texts_to_sequences(invocations)
cmd_sequences = cmd_tokenizer.texts_to_sequences(commands)
'''

In [ ]:
'''
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_input_length = max([len(seq) for seq in inv_sequences] + [len(seq) for seq in inv_sequences])
max_output_length = max([len(seq) for seq in cmd_sequences] + [len(seq) for seq in cmd_sequences])

inv_padded = pad_sequences(inv_sequences, maxlen=max_input_length, padding='post')
cmd_padded = pad_sequences(cmd_sequences, maxlen=max_output_length, padding='post')
'''

In [ ]:
'''
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(inv_padded, cmd_padded, test_size=0.1)

'''


In [ ]:
'''
input_vocab_size = len(inv_tokenizer.word_index) + 1
output_vocab_size = len(cmd_tokenizer.word_index) + 1
latent_dim = 128  # Adjust this to the desired size of LSTM/GRU hidden state
'''

In [ ]:
'''
input_vocab_size
'''

In [ ]:
'''
from tensorflow.keras.layers import Embedding
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding

embedding_dim = 128  # Adjust this to the desired size of the embeddings

# Define input layer
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)

# Add LSTM layer for the encoder
encoder_lstm = LSTM(latent_dim, return_state=True)
_, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embedding)

# Save the encoder states for use in the decoder
encoder_states = [encoder_state_h, encoder_state_c]
'''

In [ ]:
'''
# Define input layer
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(output_vocab_size, embedding_dim)(decoder_inputs)

# Add LSTM layer for the decoder
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Add a Dense layer to generate output tokens
decoder_dense = Dense(output_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
'''

In [ ]:
'''
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
'''

In [ ]:
'''
decoder_target_data = np.zeros((len(train_y), max_output_length, output_vocab_size), dtype='float32')
for i, seq in enumerate(train_y):
    for j, word in enumerate(seq):
        if j > 0:  # Ignore the first padding element
            decoder_target_data[i, j - 1, word] = 1.0
            '''

In [ ]:
'''
# Set the training parameters
batch_size = 8
epochs = 10
validation_split = 0.2

# Train the model
history = model.fit(
    [train_x, train_y], decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split
)
'''


In [ ]:
'''
# Define the encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Define the decoder model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

'''

In [ ]:
'''
def predict(input_seq):
    # Encode the input sequence to get the encoder states
    encoder_states_value = encoder_model.predict(input_seq)

    # Generate an empty target sequence
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = cmd_tokenizer.word_index['startseq']

    # Loop through each step of the output sequence
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + encoder_states_value
        )

        # Sample the most probable token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = cmd_tokenizer.index_word[sampled_token_index]
        decoded_sentence.append(sampled_word)

        # Check for the stop condition
        if sampled_word == 'endseq' or len(decoded_sentence) > max_output_length:
            stop_condition = True

        # Update the target sequence and states
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        encoder_states_value = [h, c]

    return ' '.join(decoded_sentence[:-1])  # Remove <eos> from the output
'''

In [ ]:
'''
for i in range(10):  # Change this to the number of examples you want to test
    input_seq = test_x[i:i+1]
    decoded_sentence = predict(input_seq)
    print('Input:', ' '.join([inv_tokenizer.index_word[w] for w in input_seq[0] if w != 0]))
    print('Predicted command:', decoded_sentence)
    print('True command:', ' '.join([cmd_tokenizer.index_word[w] for w in test_y[i] if w != 0]))
    print('\n')
    '''